In [ ]:
import os
import re
from openai import OpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

# 🔑 환경 설정
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"  # ← 실제 키로 교체
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [21]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)

In [22]:
# ✅ 한-영 교수 이름 매핑
professor_name_map = {
    "노맹석": "Maengseok Noh",
    "문형빈": "HyungBin Moon",
    "하지환": "Jihwan Ha",
    "지준화": "Junhwa Chi",
}

In [23]:
# ✅ 번역
def translate_with_gpt(text, source_lang="ko", target_lang="en") -> str:
    prompt = f"Translate this from {source_lang} to {target_lang}:\n\n{text}"
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

In [24]:
# ✅ 질문 유형 분류
def classify_question_type(question_ko: str) -> str:
    prompt = f"""
다음 질문의 유형을 아래 중 하나로 분류해 주세요:
- 논문_목록
- 논문_요약
- 연구_흐름

질문: {question_ko}
질문 유형:"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt.strip()}],
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [25]:
translate_with_gpt("문형빈 교수님의 논문 목록은")

"The list of Professor Moon Hyung-bin's papers is"

In [26]:
# ✅ 교수명 추출
def extract_professor_name(question: str) -> str | None:
    match = re.search(r"([가-힣]{2,4})\s*교수", question)
    return match.group(1) if match else None

In [27]:
def get_first_page_summary(doc: Document) -> str:
    title = doc.metadata.get("title", "제목 정보 없음")
    content = doc.page_content.strip().split("\n")[:2]  # 첫 줄만
    return f"📌 제목: {title}\n📄 요약: {content}"


In [28]:
def format_doc_with_metadata(doc: Document) -> str:
    professor = doc.metadata.get("professor", "교수 정보 없음")
    title = doc.metadata.get("title", "제목 정보 없음")
    content = doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content
    return f"""🧑‍🏫 교수: {professor}
📄 내용 요약:
{content}
"""

# ✅ 프롬프트 템플릿
prompt_templates = {
    "논문_목록": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are provided with a collection of academic papers written by a professor. 
Based on the following user request, list the key papers along with:

1. The title of each paper (📌 Please keep the title in English)  
2. The publication year (if available)  
3. A few core keywords representing the main topic (in Korean)  
4. The author(s) of each paper (in Korean)

User question:
{question}

Paper content:
{context}

📌 Please write your response in Korean using a respectful and organized tone, **but keep the paper titles in English**.

논문 목록 요약 (in Korean):"""
    ),
    "논문_요약": PromptTemplate(
        input_variables=["context"],
        template="""
You are a research summarization assistant. Based on the following academic paper, provide a clear and concise summary including the following elements:

1. Research subject (what or who is being studied)  
2. Research method (how it was studied)  
3. Research findings (what was discovered)  
4. Suggestions or implications (recommendations or conclusions)

Paper content:
{context}

📌 Please write your summary in Korean, using a polite and professional tone.

논문 요약 (in Korean):"""
    ),
    "연구_흐름": PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are an academic assistant. Given a collection of research papers written by a single professor, analyze how the research topics or areas of interest have evolved over time. 
Identify key shifts, trends, or patterns chronologically based on the publication content.

Context:
{context}

Question:
{question}

Now, summarize the chronological progression of the professor’s research focus. 
📌 Please write your answer in Korean using a clear and respectful tone.

연구 흐름 요약 (한국어로):"""
    )
}

In [29]:
def process_question(question_ko: str):
    # 1. 질문 유형 분류
    question_type = classify_question_type(question_ko)

    # 2. 교수 이름 추출
    target_author_ko = extract_professor_name(question_ko)
    target_author_en = professor_name_map.get(target_author_ko) if target_author_ko else None

    if question_type in ["논문_목록", "연구_흐름"] and not target_author_en:
        raise ValueError("질문에서 유효한 교수 이름을 찾을 수 없습니다.")

    # 3. 질문 번역
    question_en = translate_with_gpt(question_ko)

    # 4. 검색 (professor 기준 수동 필터링)
    collection = db3._collection.get(include=["metadatas", "documents"])

    # metadatas와 documents를 묶어서 Document 객체로 재구성
    docs = [
        Document(page_content=page, metadata=meta)
        for page, meta in zip(collection["documents"], collection["metadatas"])
        if meta.get("professor") == target_author_en
    ]

    # 5. 첫 페이지 기반 요약용 context 구성
    context_text = "\n\n---\n\n".join(get_first_page_summary(doc) for doc in docs)

    # 5. context 구성
    if question_type == "논문_목록":
        context_text = "\n\n---\n\n".join(
            f"📌 제목: {doc.metadata.get('title', '제목 정보 없음')}\n📄 요약: {doc.page_content.strip().split('\n')[0]}"
            for doc in docs
        )
        prompt = prompt_templates["논문_목록"]

    elif question_type == "연구_흐름":
        context_text = "\n\n---\n\n".join(get_first_page_summary(doc) for doc in docs)
        prompt = prompt_templates["연구_흐름"]

    else:  # 논문_요약
        context_text = "\n\n---\n\n".join(doc.page_content for doc in docs)
        prompt = prompt_templates["논문_요약"]



    # 7. 실행
    chain = prompt | ChatOpenAI(model="gpt-4o")
    inputs = {"context": context_text}
    if "question" in prompt.input_variables:
        inputs["question"] = question_ko

    result = chain.invoke(inputs)

    print(f"[{question_type.upper()}]")
    print(result.content)


In [30]:
# ✅ 실행 예시
question_ko = "노맹석 교수님의 논문 정리해줘"
process_question(question_ko)

[논문_목록]
노맹석 교수님의 논문을 다음과 같이 정리하였습니다:

1. **Dispersion frailty models and HGLMs**
   - **발행연도**: 정보 없음
   - **핵심 키워드**: 분산 취약성 모델, HGLM, 통계학 (분산 취약성 모델, HGLM, 통계학)
   - **저자**: 노맹석, 하인덕, 이영조

2. **Extended negative binomial hurdle models**
   - **발행연도**: 정보 없음
   - **핵심 키워드**: 부정 이항 모델, 과산포, 제로 카운트 (부정 이항 모델, 과산포, 제로 카운트)
   - **저자**: 정보 없음

3. **Hierarchical likelihood approach to non-Gaussian factor analysis**
   - **발행연도**: 정보 없음
   - **핵심 키워드**: 비가우시안 인자 분석, 계층적 우도, 통계 시뮬레이션 (비가우시안 인자 분석, 계층적 우도, 통계 시뮬레이션)
   - **저자**: 노맹석 외

4. **Hierarchical likelihood methods for nonlinear and generalized linear mixed models with missing data and measurement errors in covariates**
   - **발행연도**: 정보 없음
   - **핵심 키워드**: 비선형 모델, 일반화 선형 혼합 모델, 누락 데이터 (비선형 모델, 일반화 선형 혼합 모델, 누락 데이터)
   - **저자**: 노맹석 외

5. **Hierarchical-likelihood approach for nonlinear mixed-effects models**
   - **발행연도**: 2008
   - **핵심 키워드**: 비선형 혼합효과 모델, 계층적 우도, 바이어스 감소 (비선형 혼합효과 모델, 계층적 우도, 바이어스 감소)
   - **저자**: 노맹석, 이영조

6. **Mul

In [31]:
process_question('노맹석 교수님의 연구 동향')

[연구_흐름]
노맹석 교수님의 연구는 주로 통계 모델링 및 분석 방법론에 집중되어 있으며, 특히 계층적 가능도(hierarchical likelihood)와 관련된 여러 분야에 걸쳐 다양한 연구를 진행해왔습니다. 다음은 연구 주제의 시간적 발전을 요약한 것입니다.

1. **초기 연구 (2006년 이전)**: 노맹석 교수님은 이 시기에 주로 산포 취약성 모델과 HGLMs(Hierarchical Generalized Linear Models)에 대한 연구를 진행하였습니다. 이 연구들은 주로 통계 모델링에서의 이질성 감지 및 모델링에 중점을 두었습니다.

2. **중기 연구 (2006년~2012년)**: 이 시기에는 계층적 가능도 접근법을 다양한 통계 모델에 적용하는 연구가 주를 이루었습니다. 예를 들어, 비선형 및 일반화 선형 혼합 모델(NLMMs, GLMMs)에서의 계층적 가능도 방법론을 제안하고, 결측 데이터와 공변량의 측정 오류가 있는 경우에도 효율적인 추론 방법을 개발하였습니다.

3. **최근 연구 (2012년 이후)**: 최근에는 강건한 추정 방법을 적용하여 드롭아웃 모델과 같은 복잡한 데이터 구조를 다루는 연구를 진행하고 있습니다. 이 연구들은 특히 관측되지 않은 데이터나 결측 데이터가 존재하는 경우에도 정확한 추정을 가능하게 하는 방법론 개발에 중점을 두고 있습니다. 또한, 계층적 가능도 방법을 통해 비정규 분포에서도 안정적인 추정을 할 수 있는 방법을 탐구하고 있습니다.

노 교수님의 연구는 전체적으로 통계 모델링의 정밀성과 효율성을 높이는 데 기여하고 있으며, 특히 복잡한 데이터 구조를 다루는 데 있어 매우 중요한 역할을 하고 있습니다. 이러한 연구들은 통계학 분야에서의 다양한 실질적 문제 해결에 기여하고 있습니다.


In [33]:
process_question('Hierarchical likelihood methods for nonlinear and generalized linear mixed models with missing data and measurement errors in covariates논문 정리해줘')

[논문_요약]
논문 요약:

1. 연구 주제: 이 연구는 도시 내 녹지 공간이 주민들의 정신 건강에 미치는 영향을 조사하였습니다.

2. 연구 방법: 연구진은 대규모 도시 거주자를 대상으로 설문 조사를 실시하여 녹지 공간 접근성과 정신 건강 상태 간의 상관관계를 분석하였습니다. 또한, 위성 이미지를 통해 도시 내 녹지 공간의 비율을 정량적으로 측정하였습니다.

3. 연구 결과: 연구 결과, 녹지 공간에 대한 높은 접근성을 가진 주민들이 그렇지 않은 주민들에 비해 더 긍정적인 정신 건강 상태를 보고하였습니다. 특히, 스트레스 수준과 우울증 증상이 녹지 공간 접근성에 따라 유의미하게 감소하는 것으로 나타났습니다.

4. 제안 및 함의: 연구는 도시 계획자와 정책 입안자들이 주민들의 정신 건강 증진을 위해 더 많은 녹지 공간을 조성할 필요가 있음을 시사합니다. 또한, 녹지 공간의 접근성을 높이는 것이 주민들의 전반적인 삶의 질 향상에 기여할 수 있다는 결론을 내렸습니다.
